In [1]:
local = True
data_transform=True
transform_method = ""

# About this notebook
- Deberta-v3-large starter code
- pip wheels is [here](https://www.kaggle.com/code/yasufuminakama/pppm-pip-wheels)
- Training notebook is [here](https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-w-w-b-train)

If this notebook is helpful, feel free to upvote :)

In [2]:
# ====================================================
# Directory settings
# ====================================================
exp_names = [
            #  "roberta-large",
            "microsoft-mpnet-base",
            "funnel-transformer-large512",
             "deberta-v3-base",
             "microsoft-deberta-large",
             
            ]
import os
if local:
    INPUT_DIR = '../../data/us-patent-phrase-to-phrase-matching/'
    
    OUTPUT_DIR = f"./output/stacking1dcnn/"
else:
    INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
    OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

# CFG

In [4]:
# ====================================================
# CFG
# ====================================================
class CFG:
    
    #######実行ごとに変更必要の可能性あり
    EXP_NAMES = exp_names
    models=[
            # "roberta-large",
            "microsoft/mpnet-base",
            "funnel-transformer-large512",
            "microsoft/deberta-v3-base",
            "microsoft/deberta-v3-large",
            
           ]
    folder_names = [
                    # "robertalargepass1fold",
                    "mpnetbase",
                    "funnellarge512fold4",
                   "debertabase",
                   "debertalarge",
                    
                    
                  ]
    
    use_hidden_states = [
        # False, 
        False,
        False, True, False
    ]
    
    max_lens = [
        # 175, 
        125,
        125, 133, 133,]
    pass_folds_flg = False
    pass_folds  = [
        # 1, -1,
        -1, -1, -1
    ]
    
    #######
    
    paths = []
    config_paths = []
    model_paths = []
    for EXP_NAME in EXP_NAMES:
        if local:
            path=f"../exp4/output/{EXP_NAME}/"
            config_path=path+'config.pth'
            model_path=f'../exp4/output/{EXP_NAME}/'
            cpc_path = f"{INPUT_DIR}/cpc_texts.pth"
        else:
            path=f"../input/{folder_name}/{EXP_NAME}/"
            config_path=path+'config.pth'
            model_path=f'../input/{folder_name}/'
            cpc_path = "../input/pppm-deberta-v3-large-baseline-w-w-b-train/cpc_texts.pth"
        paths.append(path)
        config_paths.append(config_path)
        model_paths.append(model_path)
        
    num_workers=4
    hidden_states = []
    for model in models:
        if "small" in model:
            hidden_state  = 512
        elif "base" in model:
            hidden_state  = 768
        elif "large" in model:
            hidden_state  = 1024
        elif "xlarge" in model:
            hidden_state =  1536
        hidden_states.append(hidden_state)
    
    batch_size=32
    fc_dropout=0.2
    target_size=1
    
    seed=42
    n_fold=4
    trn_fold=[i for i in range(n_fold)]
    pass_fold = []
#     torch.load(CFG.model_path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
    

# Library

In [5]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

if not local:
    os.system('pip uninstall -y transformers')
    os.system('pip uninstall -y tokenizers')
    os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
    os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if local:
    device = torch.device("cpu")

torch.__version__: 1.9.0+cu111
tokenizers.__version__: 0.11.0
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


In [6]:
import numpy as np
import random
import pandas as pd
from copy import deepcopy as dp

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss


# Utils

In [7]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score




def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

In [8]:
# oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
# labels = oof_df['score'].values
# preds = oof_df['pred'].values
# score = get_score(labels, preds)
# LOGGER.info(f'CV Score: {score:<.4f}')
oof_dfs = pd.DataFrame()
for path in CFG.paths:
    oof_file = [f for f in os.listdir(path) if "oof_df.pkl" in f][0]
    oof_df = pd.read_pickle(path+oof_file)
    labels = oof_df['score'].values
    preds = oof_df[["id",'pred']]
    score = get_score(labels, preds["pred"])
    LOGGER.info(f'CV Score: {score:<.4f}')
    oof_dfs = pd.concat([oof_dfs, oof_df[["pred"]]],axis=1)
oof_dfs.columns = [c+str(i) for i, c in enumerate(oof_dfs.columns)]
oof_dfs["id"] = preds["id"]


CV Score: 0.8159
CV Score: 0.8485
CV Score: 0.8422
CV Score: 0.8574


In [9]:
oof_dfs

,pred0,pred1,pred2,pred3,id
0,0.030808,0.000880,0.003149,0.000215,54c1e3b9184cb5b6
1,0.276099,0.236394,0.247720,0.241680,ef2d4c2e6bbb208d
2,0.491518,0.428095,0.501632,0.503771,4c3f2750e7540ab7
3,0.506613,0.482324,0.536995,0.479786,bfd7270f57530991
4,0.448548,0.273144,0.073328,0.005082,cc96541d4987b399
...,...,...,...,...,...
36468,0.253587,0.265766,0.257354,0.249057,ede41dd2a61bb0a9
36469,0.956732,0.999820,0.794225,0.819764,4f366b6369dbfbf2
36470,0.537947,0.523277,0.581828,0.504144,51421420985d5c93
36471,0.956423,0.999939,0.998544,0.999254,8ceaa2b5c2d56250


In [10]:
# ../input/d/datasets/trevenanter/robertalarge/roberta-large/2022-04-23-20-31-47oof_df.pkl

# Data Loading

In [11]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


In [12]:
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load(CFG.cpc_path)
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

,id,anchor,target,context,context_text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE


In [13]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

,id,anchor,target,context,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[SEP]altering gas flow[SEP]MECH...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[SEP]lower locating[SEP]PERFORMI...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[SEP]upper portion[SEP]TEXTILES; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[SEP]artificial neural netwo...


# tokenizer

In [14]:
# # ====================================================
# # tokenizer
# # ====================================================
# # CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
# # CFG.tokenizer = AutoTokenizer.from_pretrained("../input/roberta-large-tokenizer/tokenizer/", use_fast=False)

# CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+"tokenizer/",use_fast=False)
# # CFG.path+'tokenizer/'
# # AutoTokenizer.from_pretrained('../input/robertalarge/roberta-large/2022-04-23-20-31-47tokenizer')

In [15]:
# AutoTokenizer.from_pretrained('../input/robertalarge/roberta-large/2022-04-23-20-31-47tokenizer', use_fast=True)

# Dataset

In [16]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text, tokenizer, max_len):
    inputs = tokenizer(text,
                           add_special_tokens=True,
                           max_length=max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item], self.tokenizer, self.max_len)
        return inputs

# Model

In [17]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, hidden_state=False, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, hidden_state),
            nn.Tanh(),
            nn.Linear(hidden_state, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [18]:
class CustomModel_512(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

# inference

In [19]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

## make staking data

In [20]:
all_predictions = pd.DataFrame()
for exp_num in range(len(CFG.EXP_NAMES)):
    # if exp_num in [0,1,] and local:continue
    print(f"=========start {CFG.EXP_NAMES[exp_num]}=========")
    CFG.tokenizers = []
    tokenizer_path = [d for d in os.listdir(CFG.paths[exp_num]) if "tokenizer" in d]
    if len(tokenizer_path)==0:
        
        path = "../input/pppm-deberta-v3-large-baseline-w-w-b-train/"
        tokenizer_path = ["tokenizer/"]
    else:
        path = CFG.paths[exp_num]
    tokenizer = AutoTokenizer.from_pretrained(path+f"{tokenizer_path[0]}/",use_fast=False)

    test_dataset = TestDataset(CFG, test, tokenizer=tokenizer, max_len=CFG.max_lens[exp_num])
    test_loader = DataLoader(test_dataset,
                             batch_size=CFG.batch_size,
                             shuffle=False,
                             num_workers=CFG.num_workers, pin_memory=False, drop_last=False)
    predictions = []
    for fold in CFG.trn_fold:
        if CFG.pass_folds_flg:
            if fold == CFG.pass_folds[exp_num]:
                continue
        if CFG.use_hidden_states[exp_num]:
            model = CustomModel(CFG, config_path=CFG.config_paths[exp_num],hidden_state=CFG.hidden_states[exp_num] , pretrained=False)
            if local:
                state = torch.load(CFG.model_paths[exp_num]+f"{CFG.models[exp_num].replace('512','').replace('/', '-')}_fold{fold}_best.pth",
                               map_location=torch.device('cpu'))
            else:
                state = torch.load(CFG.model_paths[exp_num]+f"{CFG.EXP_NAMES[exp_num]}/{CFG.models[exp_num].replace('512','').replace('/', '-')}_fold{fold}_best.pth",
                               map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
        else:
            model = CustomModel_512(CFG, config_path=CFG.config_paths[exp_num],  pretrained=False)
            if local:
                state = torch.load(CFG.model_paths[exp_num]+f"{CFG.models[exp_num].replace('512','').replace('/', '-')}_fold{fold}_best.pth",
                               map_location=torch.device('cpu'))
            else:
                state = torch.load(CFG.model_paths[exp_num]+f"{CFG.EXP_NAMES[exp_num]}/{CFG.models[exp_num].replace('512','').replace('/', '-')}_fold{fold}_best.pth",
                               map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        predictions.append(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = np.mean(predictions, axis=0)
    all_predictions = pd.concat([all_predictions,pd.DataFrame(predictions)], axis=1)

=========start microsoft-mpnet-base=========


  0%|          | 0/2 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

  0%|          | 0/2 [00:00<?, ?it/s]

Exception ignored in: <function _ConnectionBase.__del__ at 0x7f6c1d333170>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


...

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times



  0%|          | 0/2 [00:00<?, ?it/s]

Exception ignored in: <function _ConnectionBase.__del__ at 0x7f6c1d333170>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


  0%|          | 0/2 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

# 1DCNN data

In [21]:
all_predictions.columns = [f"pred{i}" for i in range(len(all_predictions.columns))]
all_predictions

,pred0,pred1,pred2,pred3
0,0.567070,0.500501,0.639312,0.559708
1,0.704946,0.780471,0.780287,0.755372
2,0.369264,0.432684,0.574027,0.467593
3,0.280511,0.298164,0.328616,0.228775
4,0.069939,0.001921,0.001656,0.000822
5,0.518947,0.513538,0.545877,0.488860
6,0.440710,0.454443,0.467361,0.478257
7,0.031812,0.000055,0.000760,0.000091
8,0.269882,0.303811,0.407280,0.293604
9,0.956727,0.999937,0.997505,0.999379


In [22]:
class oneDCNN(nn.Module):

        def __init__(self, num_features, num_targets, hidden_size):
            super(oneDCNN, self).__init__()
            cha_1 = 256
            cha_2 = 512
            cha_3 = 512

            cha_1_reshape = int(hidden_size/cha_1)
            cha_po_1 = int(hidden_size/cha_1/2)
            cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

            self.cha_1 = cha_1
            self.cha_2 = cha_2
            self.cha_3 = cha_3
            self.cha_1_reshape = cha_1_reshape
            self.cha_po_1 = cha_po_1
            self.cha_po_2 = cha_po_2

            self.batch_norm1 = nn.BatchNorm1d(num_features)
            self.dropout1 = nn.Dropout(0.1)
            self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

            self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
            self.dropout_c1 = nn.Dropout(0.1)
            self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

            self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

            self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2 = nn.Dropout(0.1)
            self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_1 = nn.Dropout(0.3)
            self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_2 = nn.Dropout(0.2)
            self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

            self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

            self.flt = nn.Flatten()

            self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
            self.dropout3 = nn.Dropout(0.2)
            self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        def forward(self, x):

            x = self.batch_norm1(x)
            x = self.dropout1(x)
            x = F.celu(self.dense1(x), alpha=0.06)

            x = x.reshape(x.shape[0],self.cha_1,
                          self.cha_1_reshape)

            x = self.batch_norm_c1(x)
            x = self.dropout_c1(x)
            x = F.relu(self.conv1(x))

            x = self.ave_po_c1(x)

            x = self.batch_norm_c2(x)
            x = self.dropout_c2(x)
            x = F.relu(self.conv2(x))
            x_s = x

            x = self.batch_norm_c2_1(x)
            x = self.dropout_c2_1(x)
            x = F.relu(self.conv2_1(x))

            x = self.batch_norm_c2_2(x)
            x = self.dropout_c2_2(x)
            x = F.relu(self.conv2_2(x))
            x =  x * x_s

            x = self.max_po_c2(x)

            x = self.flt(x)

            x = self.batch_norm3(x)
            x = self.dropout3(x)
            x = self.dense3(x)

            return x

   

In [23]:
class StackingDataset(Dataset):
    def __init__(self, df, features, labels=None):
        self.df = df[features].values
        if labels is not None:
            self.labels = labels.values
        
    def __len__(self, ):
        return len(self.df)
    
    
    def __getitem__(self, item):
        inputs =  torch.FloatTensor(self.df[item]).float()
        
        if not hasattr(self, 'labels'):
            return inputs
        labels = torch.tensor(self.labels[item]).float()
        return inputs, labels
    

In [24]:
def inference_fn(test_loader, model, device):

    model.eval()
    preds = []

    for step, (x) in enumerate(test_loader):

        x = x.to(device, non_blocking=True)

        with torch.no_grad():
            pred = model(x)

        preds.append(pred.sigmoid().detach().cpu().numpy())
#         preds.append(pred.tanh().detach().cpu().numpy())

    preds = np.concatenate(preds)

    return preds

In [25]:
class oneDCNNCFG:
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=1024
    dropout=0.3
    lr=1e-4
    batch_size=256
    epochs=50
    weight_decay=1e-5
    n_fold = 5
    features = oof_dfs.columns[:-1]

In [26]:
from tqdm.notebook import tqdm

In [27]:



class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [28]:
def run_single_nn(cfg, test, features, device, fold_num, seed, MODEL,save_dir, model_name):
    seed_everything(seed=seed)
    test_dataset = StackingDataset(test, features, None)
    
    test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, 
                              num_workers=4, pin_memory=False, drop_last=False)
    
    model = MODEL(
            num_features=len(features), num_targets=1, hidden_size=cfg.hidden_size
    )
    
    model.load_state_dict(torch.load(f"{save_dir}{model_name}_fold{fold_num}_seed{seed}.pth"))
    model.to(device, non_blocking=True)
    # inference
    predictions = inference_fn(test_loader, model, device)     
    return predictions
                          

def run_kfold_nn(cfg, test, features, device, n_fold, seed,MODEL, save_dir, model_name):
    oof = np.zeros((len(test), 1))
    for fold_num in range(n_fold):
        LOGGER.info(f"fold {fold_num}")
        _oof = run_single_nn(cfg, test, features, device, fold_num, seed, MODEL,save_dir, model_name)
        oof += _oof
    return oof/n_fold

In [29]:

oof = np.zeros((len(test), 1))
SEED = [42, 1999, 2022]
for i, seed in enumerate(SEED):
    print(seed)
    _oof = run_kfold_nn(oneDCNNCFG, all_predictions, all_predictions.columns, device,
                        n_fold=5, seed=seed, MODEL=oneDCNN, save_dir=OUTPUT_DIR,
                        model_name="1dcnn")
    oof += _oof/len(SEED)
    

fold 0


42


RuntimeError: Error(s) in loading state_dict for oneDCNN:
	size mismatch for batch_norm1.weight: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for batch_norm1.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for batch_norm1.running_mean: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for batch_norm1.running_var: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for dense1.weight_v: copying a param with shape torch.Size([1024, 3]) from checkpoint, the shape in current model is torch.Size([1024, 4]).

In [ ]:
submission["score"] = oof
submission
# test[["id", "score"]].to_csv(f"submission.cvs",index=False)

In [ ]:
# if not local:
#     submission['score'] = predictions
#     display(submission.head())
#     submission[['id', 'score']].to_csv('submission.csv', index=False)

In [ ]:
# all_predictions

In [ ]:
torch.tensor(np.arange(128)).unsqueeze().shape